In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-08-21 14:05:45.401121: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 14:05:45.401289: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 14:05:45.640881: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_dir = '/kaggle/input/emotion-detection-fer/train'
test_dir = '/kaggle/input/emotion-detection-fer/test'

In [3]:
# PREPROCESSING DATASET

train_datagen = ImageDataGenerator(
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest',
    rescale = 1./255,
    validation_split = 0.2
)

val_datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2
)

In [4]:
# LOADING DATASET AND SENDING FOR PREPROCESSING

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size = 64,
    color_mode='rgb',
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed = 123
)

val_generator = val_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size = 64,
    color_mode='rgb',
    class_mode='categorical',
    subset='validation',
    shuffle=True,
    seed = 123
)

test_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    batch_size = 64,
    color_mode='rgb',
    class_mode='categorical',
)

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [5]:
# # DATA VISUALISATION

# def display_images_with_labels(dataset, class_names):
#     # Take a batch of 9 images from the dataset
#     plt.figure(figsize=(10, 10))
#     for images, labels in dataset.take(1):
#         for i in range(9):
#             print(images[i].numpy().min(), images[i].numpy().max())
#             ax = plt.subplot(3, 3, i + 1)
#             plt.imshow(images[i].numpy().astype("uint8"))
#             plt.title(class_names[labels[i]])
#             plt.axis("off")
#     plt.show()

# # Assuming you have a list of class names and a dataset
# class_names = train_dataset.class_names  # Example class names, adjust as needed
# display_images_with_labels(train_dataset, class_names)


In [6]:
def feature_extractor(inputs):
    feature_extractor = tf.keras.applications.DenseNet169(input_shape=(48,48, 3),
                                               include_top=False,
                                               weights='/kaggle/input/densenet169-weights/densenet169_weights_tf_dim_ordering_tf_kernels_notop.h5')(inputs)
    
    return feature_extractor

def classifier(inputs):
    x = tf.keras.layers.GlobalMaxPool2D()(inputs)
#     x = tf.keras.layers.Dense(256, activation="relu", kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
#     x = tf.keras.layers.Dropout(0.3)(x)
#     x = tf.keras.layers.Dense(1024, activation="relu", kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
#     x = tf.keras.layers.Dropout(0.5)(x)
#     x = tf.keras.layers.Dense(512, activation="relu", kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
#     x = tf.keras.layers.Dropout(0.5) (x)
    x = tf.keras.layers.Dense(32, activation="relu", kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(16, activation="relu", kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(7, activation="softmax")(x)
    return x

def final_model(inputs):
    densenet_feature_extractor = feature_extractor(inputs)
    classification_output = classifier(densenet_feature_extractor)
    
    return classification_output

def compile_model():
    inputs = tf.keras.Input(shape=(48,48, 3))
    classification_output = final_model(inputs)
    model = tf.keras.Model(inputs=inputs,outputs=classification_output)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=['Accuracy']
    )
    return model

In [7]:
model = compile_model()

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 48, 48, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ densenet169 (Functional)        │ (None, 1, 1, 1664)     │    12,642,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling2d            │ (None, 1664)           │             0 │
│ (GlobalMaxPooling2D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │        53,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 7)              │           119 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,696,807 (48.43 MB)

 Trainable params: 12,538,407 (47.83 MB)

 Non-trainable params: 158,400 (618.75 KB)

In [8]:
model.layers[1].trainable = False

earlyStoppingCallback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    verbose=1,
    restore_best_weights=True,
)

history = model.fit(x=train_generator,
                    validation_data=val_generator,
                    epochs=10,
                    callbacks=earlyStoppingCallback,
                    )

history = pd.DataFrame(history.history)

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


359/359 ━━━━━━━━━━━━━━━━━━━━ 252s 624ms/step - Accuracy: 0.2339 - loss: 3.4051 - val_Accuracy: 0.2513 - val_loss: 1.8226
Epoch 2/10
359/359 ━━━━━━━━━━━━━━━━━━━━ 149s 414ms/step - Accuracy: 0.2522 - loss: 1.8160 - val_Accuracy: 0.2513 - val_loss: 1.8130
Epoch 3/10
359/359 ━━━━━━━━━━━━━━━━━━━━ 148s 411ms/step - Accuracy: 0.2484 - loss: 1.8157 - val_Accuracy: 0.2513 - val_loss: 1.8161
Epoch 4/10
359/359 ━━━━━━━━━━━━━━━━━━━━ 143s 398ms/step - Accuracy: 0.2536 - loss: 1.8147 - val_Accuracy: 0.1730 - val_loss: 1.8332
Epoch 5/10
359/359 ━━━━━━━━━━━━━━━━━━━━ 141s 390ms/step - Accuracy: 0.2427 - loss: 1.8216 - val_Accuracy: 0.2513 - val_loss: 1.8271
Epoch 5: early stopping
Restoring model weights from the end of the best epoch: 2.
